<a href="https://colab.research.google.com/github/LidiaISKIN/ISKIN/blob/master/MDPI_Vitkova_Chechulin_Kotenko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

The data sets are located at: https://www.kaggle.com/lidiaiskin/sources-of-information-in-social-network-vk-2021

In [ ]:
DF = pd.read_csv('./DF_june.csv', sep= ';')

In [ ]:
DF.head(10)

,Source,Post,ID ath,Data,type,like,comm,reposts,views
0,-10174752,-10174752,-10174752,1622953721,1,5,0,0,566
1,-101762028,-101762028,-101762028,1624454958,1,16,0,2,612
2,2658939,-101373101,572362949,1623741748,"0,5",4,0,0,0
3,-101373101,-101373101,-101373101,1623738600,1,193,14,18,4401
4,-101443126,-101443126,-101443126,1624939272,1,12,0,2,1307
5,-101555177,-101555177,-101555177,1623917621,1,118,11,14,9935
6,456500,-101555177,30637949,1624879701,"0,5",1,0,0,0
7,-101555177,-101555177,-101555177,1624818610,1,552,45,36,17096
8,-106770162,-106770162,-106770162,1622562270,1,2,0,1,98
9,-101982925,-101982925,-101982925,1623265468,1,288,69,109,50157


In [ ]:
DF.columns

Index(['Source', 'Post', 'ID ath', 'Data', 'type', 'like', 'comm', 'reposts',
       'views'],
      dtype='object')

In [ ]:
DF.loc[:, ['Source', 'Post', 'ID ath', 'Data', 'type', 'like', 'comm', 'reposts',
       'views']].head(10)

,Source,Post,ID ath,Data,type,like,comm,reposts,views
0,-10174752,-10174752,-10174752,1622953721,1,5,0,0,566
1,-101762028,-101762028,-101762028,1624454958,1,16,0,2,612
2,2658939,-101373101,572362949,1623741748,"0,5",4,0,0,0
3,-101373101,-101373101,-101373101,1623738600,1,193,14,18,4401
4,-101443126,-101443126,-101443126,1624939272,1,12,0,2,1307
5,-101555177,-101555177,-101555177,1623917621,1,118,11,14,9935
6,456500,-101555177,30637949,1624879701,"0,5",1,0,0,0
7,-101555177,-101555177,-101555177,1624818610,1,552,45,36,17096
8,-106770162,-106770162,-106770162,1622562270,1,2,0,1,98
9,-101982925,-101982925,-101982925,1623265468,1,288,69,109,50157


In [ ]:
DF_out = pd.DataFrame({
     'id_source': [ ],
     'sum_url':[],
     'potential_source':[],
     'index_activ':[],
     'index_view':[],
     'impact_index':[]
})

### The first algorithm. Calculation of the potential

Calculate the sum of messages from one source and add a column source_count_mess (taking into account "the type") output as a separate list  
source potential = 1 (low) if the number of messages in the list is less than the average  
cut off those with low potential, calculate a new average  
source potential = 3(high), if it has a number of messages above the second average  
else source potential = 2(medium)  
add the source potential columns to pd.DF- fill them from the list for each row

In [ ]:
Source= DF['Source'].tolist()
Type= DF['type'].tolist()

In [ ]:
Source_unic=set(Source)
Source_unic=list(Source_unic)
Source_poten=[0]*len(Source_unic)

In [ ]:
for i in range(len(Source_unic)):
    for j in range(len(Source)):
        if Source_unic[i]==Source[j]:
            if Type[j]==1:
                Source_poten[i]=Source_poten[i]+1
            else:  Source_poten[i]=Source_poten[i]+0.5

In [ ]:
sr_source=np.mean(Source_poten)
for i in range(len(Source_poten)):
    if Source_poten[i]<sr_source:
        Source_poten[i]=1

In [ ]:
Count=0
Sum=0
for i in range(len(Source_poten)):
    if Source_poten[i]>1:
        Count=Count+1
        Sum=Sum+Source_poten[i]
        sr_source=Sum/Count

In [ ]:
for i in range(len(Source_poten)):
    if Source_poten[i]>sr_source:
        Source_poten[i]=3
    if Source_poten[i]>1 and Source_poten[i]<3:
        Source_poten[i]=2

In [ ]:
dictionary = dict(zip(Source_unic, Source_poten))

### The second algorithm. Calculation of the index of activity and viewability of the source

For the source, sum up all the likes was received for all sourse' messages, all comments, all reposts and divide by the number of messages. This is the source activity index
For each source, sum up all the views to its messages and divide by the number of messages - it is the index of the source's viewability (viewing)
Form a DF_source and keep the index= id_source in it, the columns are the source potential, the source activity index, the source viewability index  

In [ ]:
Source_count_mess=[0]*len(Source_unic)
for i in range(len(Source_unic)):
    for j in range(len(Source)):
        if Source_unic[i]==Source[j]:
                Source_count_mess[i]=Source_count_mess[i]+1

In [ ]:
Post= DF['Post'].tolist()
Index_source=[0]*len(Source_unic)
for i in range(len(Post)):
    for j in range(len(Source_unic)):
        if Source[i]==Source_unic[j]:
            Index_source[j]=(Index_source[j]+DF['like'][i]+DF['comm'][i]+DF['reposts'][i])/Source_count_mess[j]
dictionary = dict(zip(Source_unic,Index_source))

In [ ]:
View_source=[0]*len(Source_unic)
for i in range(len(Source)):
    for j in range(len(Source_unic)):
        if Source[i]==Source_unic[j]:
            View_source[j]=(View_source[j]+DF['views'][i])/Source_count_mess[j]
dictionary1 = dict(zip(Source_unic,View_source))

In [ ]:
DF_out_source = pd.DataFrame({
     'id_source': [ ],
     'sum_url':[],
     'potential_source':[],
     'index_activ':[],
     'index_view':[],
     
})
DF_out_source['id_source']=Source_unic
DF_out_source['sum_url']= Source_count_mess
DF_out_source['potential_source']=Source_poten
DF_out_source['index_activ']=Index_source
DF_out_source['index_view']=View_source

DF_out_source.loc[:, ['id_source', 'sum_url', 'potential_source', 'index_activ', 'index_view']].head(10)

### The third algorithm. Calculation of the influence of the source

For each source, we normalize all indexes in each dataframe

In [3]:
x_array = np.array(Index_source)
Index_source1 = preprocessing.normalize([x_array])
x_array = np.array(View_source)
View_source1 = preprocessing.normalize([x_array])

NameError: ignored

In [ ]:
DF_out_source = pd.DataFrame({
     'id_source': [ ],
     'sum_url':[],
     'potential_source':[],
     'index_activ':[],
     'index_view':[],
     
})
DF_out_source['id_source']=Source_unic
DF_out_source['sum_url']= Source_count_mess
DF_out_source['potential_source']=Source_poten
DF_out_source['index_activ']=Index_source1[0]
DF_out_source['index_view']=View_source1[0]

DF_out_source.loc[:, ['id_source', 'sum_url', 'potential_source', 'index_activ', 'index_view']].head(10)

In [ ]:
Influence=[0]*len(Source_unic)
for i in range(len(Source_unic)):
    Influence[i]=Index_source1[0][i]*Source_poten[i]
DF_out_source['Influence'] = np.array(Influence)
DF_out_source.loc[:, ['id_source', 'sum_url', 'potential_source', 'index_activ', 'index_view','Influence']].head(10)

Для расчета влиятельности источника мы перемножаем индекс активности источника, индекс просматриваемости, потенциал  
Для расчета влиятельности автора мы перемножаем индекс активности источника, индекс просматриваемости, потенциал  
Добавляем в выходные данные столбец с индексом влиятельности по каждой строке в каждый датафрейм  

In [ ]:
Influence=[0]*len(Source_unic)
for i in range(len(Source_unic)):
    Influence[i]=Index_source1[0][i]*(Source_poten[i]+View_source1[0][i])
DF_out_source['Influence'] = np.array(Influence)
DF_out_source.loc[:, ['id_source', 'sum_url', 'potential_source', 'index_activ', 'index_view','Influence']].head(10)

In [ ]:
DF_out_source['id_source'] = DF_out_source['id_source'].astype(str)

In [ ]:
import scipy.stats

In [ ]:
np.quantile(DF_out_source['Influence'], q=0.99)

In [ ]:
DF_out_source['Influence'].value_counts().head(10)

In [ ]:
sorted_DF_out_source = DF_out_source.sort_values(by=['Influence' ], ascending=False)
sorted_DF_out_source.head(10)

In [ ]:
DF_out_source.columns

In [ ]:
import pickle

In [ ]:
pickle.dump(DF_out_source, open('DF_out_source.pkl', 'wb'))
pickle.dump(DF_out_ath, open('DF_out_ath.pkl', 'wb'))


In [ ]:
DF_out_ath.to_csv('out_DF_june.csv')

In [ ]:
DF_out_source.to_csv('out_DF_june_sour.csv')